In [25]:
# packages and dependencies import
import os
import sys
import json
import pandas as pd # used for data manipulation
import numpy as np # used for mathematical operations

# Used for Exploratory Data Analysis
import seaborn as sns 

# used for data visualization
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Used for data transfer and visualization 
import plotly.graph_objects as go
import plotly.io as pio
from plotly.utils import PlotlyJSONEncoder 

In [18]:
# Data Import
data = pd.read_csv("/home/tjselevani/Desktop/Apps/vscode/python/python analysis/data/last-3-months-transactions.csv")

# Convert 'created_at' column to datetime if not already
data['created_at'] = pd.to_datetime(data['created_at'])

# Extract the month from 'created_at'
data.loc[:, 'month'] = data['created_at'].dt.to_period('M').astype(str)

In [19]:
def format_months_xticks(month_labels):
    """Returns tick positions and labels for months."""
    return range(len(month_labels)), month_labels

In [20]:
data = data[data['transaction_type'] == 'CREDIT'].copy()

In [21]:
# Define unique colors for each vehicle, plus "Unknown" and "Failed Transactions"
vehicle_colors = {
    'SM191': 'blue',
    'SM192': 'green',
    'SM944': 'yellow',
    'SM055': 'purple',
    'SM024': 'orange',
    'Unknown': 'gray',
    'Failed': 'red'
}

# Map colors to vehicle_booked, filling NaN values with a default color (e.g., gray)
data.loc[:,'color'] = data['vehicle_booked'].map(vehicle_colors).fillna('gray')

# Define vehicles list (unique vehicles from the data)
vehicles = data['vehicle_booked'].dropna().unique()

In [22]:
# Aggregate data by month for each vehicle
monthly_by_vehicle = data.groupby(['month', 'vehicle_booked'])['amount'].sum().reset_index()

# Pivot the data to have vehicles as columns
monthly_pivot = monthly_by_vehicle.pivot(index='month', columns='vehicle_booked', values='amount').reset_index()

# Sort by month (assuming format is YYYY-MM)
monthly_pivot = monthly_pivot.sort_values('month')

In [26]:
def generate_monthly_earnings_chart_and_json(data, output_file):
    """
    This method generates a bar chart of monthly earnings by vehicle, preserving vehicle colors,
    saves the chart as an image using matplotlib, and saves the chart data as JSON using plotly.
    
    Parameters:
    data (pd.DataFrame): The input dataframe containing transaction data.
    output_image_path (str): The path to save the generated chart image.
    output_json_path (str): The path to save the chart data in JSON format.
    
    Returns:
    None
    """

    # Aggregate data by month for each vehicle
    monthly_by_vehicle = data.groupby(['month', 'vehicle_booked'])['amount'].sum().reset_index()

    # Reshape the data for a grouped bar chart
    monthly_melt = monthly_by_vehicle.copy()
    
    # Plotly graph data in a JSON format
    plot_data = []
    
    for vehicle in vehicle_colors.keys():
        vehicle_data = monthly_by_vehicle[monthly_by_vehicle['vehicle_booked'] == vehicle]
        plot_data.append(go.Bar(
            x=vehicle_data['month'],
            y=vehicle_data['amount'],
            name=vehicle,
            marker=dict(color=vehicle_colors.get(vehicle, 'gray'))
        ))

    # Create the plotly figure
    fig = go.Figure(data=plot_data)

    # Update layout for plotly figure
    fig.update_layout(
        title="Monthly Earnings by Vehicle",
        xaxis_title="Month",
        yaxis_title="Total Earnings (KSH)",
        barmode='stack',
        xaxis=dict(tickangle=45),
        legend_title="Vehicles"
    )

    # Create output directories
    json_dir = f"../json/all/month"
    os.makedirs(json_dir, exist_ok=True)

    json_path = f"{json_dir}/{output_file}.json"
    with open(json_path, 'w') as json_file:
        json.dump(plot_data, json_file, indent=4, cls=PlotlyJSONEncoder)

    print(f"Saved JSON Plotly data: {json_path}")

# Example usage:
data_file_path = "/home/tjselevani/Desktop/Apps/vscode/python/python analysis/data/last-3-months-transactions.csv"
output_image_path = "/home/tjselevani/Desktop/Apps/vscode/python/python analysis/files/monthly_earnings_trend_bar.png"
output_json_path = "/home/tjselevani/Desktop/Apps/vscode/python/python analysis/json/monthly_earnings_trend_bar.json"

# Generate the monthly earnings bar chart and save the image and JSON data
generate_monthly_earnings_chart_and_json(data, "monthly_earnings_trend_bar")
display_plotly_graph(output_json_path)

Saved JSON Plotly data: ../json/all/month/monthly_earnings_trend_bar.json
Error: File not found at /home/tjselevani/Desktop/Apps/vscode/python/python analysis/json/monthly_earnings_trend_bar.json
